<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=213663968" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json as js
import subprocess
import logging
import time
import shutil

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2022-05-03 00:00:00,BIOFI-USD,0.001627,0.001587,0.001706,0.001584,165278.0,0.001587
1,2022-05-04 00:00:00,BIOFI-USD,0.001587,0.001623,0.001648,0.001579,85206.0,0.001623
2,2022-05-05 00:00:00,BIOFI-USD,0.001623,0.001636,0.001673,0.001527,52331.0,0.001636
3,2022-05-06 00:00:00,BIOFI-USD,0.001636,0.001622,0.001651,0.001555,58094.0,0.001622
4,2022-05-07 00:00:00,BIOFI-USD,0.001622,0.001504,0.001643,0.001503,45516.0,0.001504
...,...,...,...,...,...,...,...,...
694590,2024-12-12 00:00:00,BRN19519-USD,0.223955,0.216795,0.230413,0.209873,548715.0,0.216795
694591,2024-12-13 00:00:00,BRN19519-USD,0.216781,0.216187,0.228582,0.214831,609011.0,0.216187
694592,2024-12-14 00:00:00,BRN19519-USD,0.216187,0.205332,0.220884,0.203732,556514.0,0.205332
694593,2024-12-15 00:00:00,BRN19519-USD,0.204812,0.201323,0.209544,0.197765,486140.0,0.201323


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = f'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'
link

'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(20)):
    url = f'https://finance.yahoo.com/markets/crypto/all/?start={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 20/20 [00:08<00:00,  2.29it/s]


In [6]:
symbol_list = []

for r in tqdm(resp_main_list):
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

100%|██████████| 20/20 [00:09<00:00,  2.21it/s]

2520


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [7]:
'''

llink1 = https://query1.finance.yahoo.com/v8/finance/chart/GLM-USD?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1=1510185600&period2=1725897789&symbol=GLM-USD&userYfid=true&lang=en-US&region=US

here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))

period2 = current_timestamp  

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1420070400&period2=1734515116


# create stocks directory

In [8]:
os.mkdir('crypto_data')

# Download all files

In [9]:
def return_timestamp(unix_timestamps, date_format='%Y-%m-%d %H:%M:%S'):
    formatted_dates = []
    for ut in unix_timestamps:
        date = datetime.fromtimestamp(ut)
        formatted_date = date.strftime(date_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

print(return_timestamp([1725898133]))

['2024-09-09 16:08:53']


In [10]:
for stock in tqdm(symbol_list):
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{stock}?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&symbol={stock}&userYfid=true&lang=en-US&region=US'
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.json", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.json")
    else:
#         print(f"Failed to download file : {stock}.json")
        pass
#     time.sleep(3)

100%|██████████| 2520/2520 [03:22<00:00, 12.45it/s]


In [11]:
symbol_list_dir = os.listdir('/kaggle/working/crypto_data/')
len(symbol_list_dir)

641

# Read a sample

In [12]:
# jsonf = pd.read_json(f'/kaggle/working/crypto_data/BTC-USD.json')
# jsondict = jsonf.to_dict()
# timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
# sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
# symbol = list(sym for s in range(len(timestamp)))
# low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
# high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
# volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
# open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
# close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
# adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
# dates = return_timestamp(timestamp)
# tempdf = pd.DataFrame(
#     {
#     'dates':dates,
#     'symbol':symbol,
#     'open':open_,
#     'close':close,
#     'high':high,
#     'low':low,
#     'volume':volume,
#     'adj_close':adj_close
#     }
# )
# tempdf

# Work on dataset

In [13]:
megadf = pd.DataFrame()

In [14]:
for jso in tqdm(symbol_list_dir):
    jsonf = pd.read_json(f'/kaggle/working/crypto_data/{jso}')
    jsondict = jsonf.to_dict()
    
    # Safely get the timestamp and check if it's None
    timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
    if timestamp is None:
        print(f"Missing timestamp for {jso}. Skipping this symbol.")
        continue  # Skip this iteration if timestamp is None
    
    # Proceed with the rest of the code after ensuring timestamp is not None
    low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
    sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
    
    # Create a list of symbols based on the length of timestamp
    symbol = list(sym for s in range(len(timestamp)))
    
    high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
    volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
    open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
    close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
    adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
    
    # Convert timestamp to dates
    dates = return_timestamp(timestamp)
    
    # Create a DataFrame
    tempdf = pd.DataFrame(
    {
        'dates': dates,
        'symbol': symbol,
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': volume,
        'adj_close': adj_close
    })
    
    # Concatenate the new data to the main DataFrame
    megadf = pd.concat([megadf, tempdf], ignore_index=True)


  3%|▎         | 22/641 [00:00<00:06, 92.45it/s] 

Missing timestamp for PUNCH33170-USD .json. Skipping this symbol.
Missing timestamp for PUFFY-USD .json. Skipping this symbol.


  7%|▋         | 42/641 [00:00<00:06, 89.13it/s]

Missing timestamp for QGOV-USD .json. Skipping this symbol.
Missing timestamp for EBULL-USD .json. Skipping this symbol.
Missing timestamp for SPEEDY33489-USD .json. Skipping this symbol.
Missing timestamp for GONDOLA-USD .json. Skipping this symbol.


 12%|█▏        | 76/641 [00:00<00:05, 101.32it/s]

...

 94%|█████████▍| 601/641 [00:12<00:01, 32.90it/s]

Missing timestamp for CBPAY-USD .json. Skipping this symbol.
Missing timestamp for DUCKY33852-USD .json. Skipping this symbol.


 98%|█████████▊| 627/641 [00:13<00:00, 31.33it/s]

Missing timestamp for LOVELY32095-USD .json. Skipping this symbol.
Missing timestamp for NCN-USD .json. Skipping this symbol.


100%|██████████| 641/641 [00:13<00:00, 46.12it/s]


In [15]:
megadf['symbol'].unique()

array(['SLOTH31163-USD', 'DIVI-USD', 'RBW-USD', 'BIP-USD', 'REV2135-USD',
       'JUSDT-USD', 'PLI-USD', 'ANKRBNB-USD', 'HABIBI31189-USD',
       'CIV11345-USD', 'CLS-USD', 'CTI-USD', 'BTS-USD', 'NUKK',
       'SWTH-USD', 'WONE-USD', 'BEFE-USD', 'EXFI-USD', 'PPT-USD',
       'BCCOIN-USD', 'WPOKT-USD', 'SHIBDOGE-USD', 'RMRK-USD', 'WITCH-USD',
       'MEE-USD', 'WVE', 'CAGA-USD', 'OAX-USD', 'AZIT-USD', 'MYST-USD',
       'PKT9523-USD', 'AREA-USD', 'FRA-USD', 'QOM-USD', 'BABY10334-USD',
       'VTC-USD', 'XTP-USD', 'VBCH-USD', 'AEG-USD', 'WOZX-USD',
       'HONK31770-USD', 'CHI-USD', 'FNCY-USD', 'PZM-USD', 'UMB-USD',
       'LCC-USD', 'ZF-USD', 'HTD-USD', 'BRWL-USD', 'CKETH-USD', 'PCX-USD',
       'WOLF30360-USD', 'GEEQ-USD', 'ORC-USD', 'LL-USD', 'QDFI-USD',
       'INTX31709-USD', 'KDAG-USD', 'WAT-USD', 'CHRP-USD', 'BNUSD-USD',
       'HAM-USD', 'EMRX-USD', 'FRM-USD', 'SSWP-USD', 'XNC-USD',
       'AVI27642-USD', 'SAFE-USD', 'HMND-USD', 'MLT-USD', 'YOOSHI-USD',
       'NVDA', 'TYPE29106-

In [16]:
first_date = megadf['dates'][0]
last_date = megadf['dates'].iloc[-1]

In [17]:
first_date,last_date

('2024-05-09 00:00:00', '2024-12-18 09:46:00')

In [18]:
megadf.dtypes

dates         object
symbol        object
open         float64
close        float64
high         float64
low          float64
volume       float64
adj_close    float64
dtype: object

In [19]:
megadf['dates'] = pd.to_datetime(megadf['dates'], format="mixed")

In [20]:
megadf.dtypes

dates        datetime64[ns]
symbol               object
open                float64
close               float64
high                float64
low                 float64
volume              float64
adj_close           float64
dtype: object

In [21]:
megadf

,dates,symbol,open,close,high,low,volume,adj_close
0,2024-05-09 00:00:00,SLOTH31163-USD,0.016607,0.023680,0.024050,0.015655,13255790.0,0.023680
1,2024-05-10 00:00:00,SLOTH31163-USD,0.023725,0.029784,0.034715,0.021040,29872494.0,0.029784
2,2024-05-11 00:00:00,SLOTH31163-USD,0.031571,0.030057,0.033531,0.025485,19258033.0,0.030057
3,2024-05-12 00:00:00,SLOTH31163-USD,0.029021,0.030869,0.031883,0.026319,13104815.0,0.030869
4,2024-05-13 00:00:00,SLOTH31163-USD,0.032345,0.051990,0.056199,0.032247,41936828.0,0.051990
...,...,...,...,...,...,...,...,...
654946,2024-12-14 00:00:00,RPG12221-USD,0.366483,0.407805,0.484760,0.365180,2664.0,0.407805
654947,2024-12-15 00:00:00,RPG12221-USD,0.407805,0.457565,0.496983,0.370527,333.0,0.457565
654948,2024-12-16 00:00:00,RPG12221-USD,0.457565,0.451944,0.471557,0.431309,567.0,0.451944
654949,2024-12-17 00:00:00,RPG12221-USD,0.451944,0.397587,0.476751,0.397208,619.0,0.397587


In [22]:
os.mkdir('dataset')

In [23]:
megadf.to_csv(f'dataset/1000_cryptos.csv',index = False)

# final dataset

In [24]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [25]:
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2024-05-09 00:00:00,SLOTH31163-USD,0.016607,0.023680,0.024050,0.015655,13255790.0,0.023680
1,2024-05-10 00:00:00,SLOTH31163-USD,0.023725,0.029784,0.034715,0.021040,29872494.0,0.029784
2,2024-05-11 00:00:00,SLOTH31163-USD,0.031571,0.030057,0.033531,0.025485,19258033.0,0.030057
3,2024-05-12 00:00:00,SLOTH31163-USD,0.029021,0.030869,0.031883,0.026319,13104815.0,0.030869
4,2024-05-13 00:00:00,SLOTH31163-USD,0.032345,0.051990,0.056199,0.032247,41936828.0,0.051990
...,...,...,...,...,...,...,...,...
654946,2024-12-14 00:00:00,RPG12221-USD,0.366483,0.407805,0.484760,0.365180,2664.0,0.407805
654947,2024-12-15 00:00:00,RPG12221-USD,0.407805,0.457565,0.496983,0.370527,333.0,0.457565
654948,2024-12-16 00:00:00,RPG12221-USD,0.457565,0.451944,0.471557,0.431309,567.0,0.451944
654949,2024-12-17 00:00:00,RPG12221-USD,0.451944,0.397587,0.476751,0.397208,619.0,0.397587


# Upload Dataset

# User secrets

In [26]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [27]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    js.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [28]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 81.9M/81.9M [00:01<00:00, 44.6MB/s]


Upload successful: 1000_cryptos.csv (82MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


In [29]:
shutil.rmtree('/kaggle/working/crypto_data')
shutil.rmtree('/kaggle/working/dataset')

# Thank you , and I will see you in next notebook :)